# Playground SQL

Pour rappel, on a 4 tables suivantes:

- Athlete : id (primary key), name, gender, team
- Event : id (primary key), location, year, distance, stroke, relay
- Team : id (primary key), event_id (foreign key), athlete_id (foreign key)
- Results : id (primary key), team_id (foreign key), results, rank, quit_reason

Le but ici va être de s'entraîner sur des requêtes SQL avancées.

In [1]:
# Connexion à la base de données
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")

# Création de la base de données
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')


GPT o1-mini, avec la connaissance du contexte, nous a concocté quelques exercices :

Voici une série de défis SQL conçus pour vous aider à vous entraîner et à maîtriser l'utilisation des Common Table Expressions (CTEs), des fonctions de fenêtre (window functions) et des classements (RANKs) dans le contexte de votre base de données sur les résultats de natation olympique.

1. Top 3 Performers par Année et Style de Nage

Écrivez une requête SQL utilisant un CTE et des fonctions de fenêtre pour identifier les trois athlètes les plus performants (les résultats les plus bas) dans chaque style de nage pour chaque année olympique.

2. Moyenne des Résultats par Équipe

Utilisez une fonction de fenêtre pour calculer le temps moyen des résultats pour chaque équipe à travers tous les événements et listez les équipes dont la performance est supérieure à la moyenne générale.

3. Amélioration des Performances au Fil du Temps

Employez un CTE pour organiser les données par athlète et année, puis utilisez des fonctions de fenêtre pour calculer la différence de résultats par rapport à l'année olympique précédente pour chaque athlète.

4. Classement de Participation aux Événements

Classez les athlètes en fonction du nombre d'événements auxquels ils ont participé en utilisant RANK ou DENSE_RANK, et listez les dix athlètes les plus actifs.

5. Performance des Équipes de Relais

Utilisez des fonctions de fenêtre pour déterminer quelles équipes de relais ont les résultats combinés les plus rapides et classez-les en conséquence.

6. Résultats Médians par Événement

Écrivez une requête utilisant des fonctions de fenêtre pour calculer le temps médian des résultats pour chaque événement et identifiez comment chaque athlète se compare à la médiane.

7. Consistance des Performances des Athlètes

Utilisez des CTE et des fonctions de fenêtre pour mesurer la consistance de chaque athlète en calculant l'écart type de leurs temps de résultats à travers différents événements, et classez-les du plus au moins constant.

8. Comparaison des Performances par Genre et Année

Utilisez des fonctions de fenêtre pour comparer les résultats moyens entre les athlètes masculins et féminins pour chaque année, et classez l'évolution des performances par genre au fil du temps.

9. Domination des Équipes par Style de Nage

Utilisez RANK pour identifier quelles équipes ont remporté le plus de médailles dans chaque catégorie de style de nage à travers toutes les années.

10. Plus Grande Amélioration des Résultats

Utilisez des CTE pour identifier les athlètes qui ont montré la plus grande amélioration dans leurs résultats au fil des Jeux Olympiques successifs.

---
Ces défis vous permettront de manipuler et d'analyser vos données de manière approfondie tout en renforçant vos compétences en SQL avancé. Bon entraînement !

## Exercice 1 : Top 3 Performers par Année et Style de Nage

"Écrivez une requête SQL utilisant un CTE et des fonctions de fenêtre pour identifier les trois athlètes les plus performants (les résultats les plus bas) dans chaque style de nage pour chaque année olympique."

On va pas compter les relais en revanche, parce qu'on a pas l'info de qui dans l'équipe a été meilleur dans le dataset.

On va aussi prendre en compte la distance, ça n'a pas de sens de comparer les résultats UNIQUEMENT par style de nage.

Voici le résultat en prenant en compte toutes les données, pas besoin de CTE ou de window function c'est trivial.

In [12]:
from sqlalchemy import text

session = engine.connect()

query = text("""
SELECT a.name, e.stroke, e.distance, e.year, r.rank
FROM athletes a
INNER JOIN teams t ON a.athlete_id = t.athlete_id
INNER JOIN events e ON t.event_id = e.event_id
INNER JOIN results r ON t.team_id = r.team_id
WHERE NOT e.is_relay
AND r.rank <= 3
AND r.rank > 0
ORDER BY e.year, e.stroke, e.distance, r.rank
""")

result = session.execute(query)
print(result.fetchall())

session.close()

[('Harry J. Hebner', 'Backstroke', 100, 1912, 1), ('Otto Fahr', 'Backstroke', 100, 1912, 2), ('Paul Kellner', 'Backstroke', 100, 1912, 3), ('Walter Bathe', 'Breaststroke', 200, 1912, 1), ('Willy Lützow', 'Breaststroke', 200, 1912, 2), ('Kurt Paul Malisch', 'Breaststroke', 200, 1912, 3), ('Walter Bathe', 'Breaststroke', 400, 1912, 1), ('Tor Henning', 'Breaststroke', 400, 1912, 2), ('Percy Courtman', 'Breaststroke', 400, 1912, 3), ('Fanny Durack', 'Freestyle', 100, 1912, 1), ('Duke Paoa Kahanamoku', 'Freestyle', 100, 1912, 1), ('Cecil Healy', 'Freestyle', 100, 1912, 2), ('Mina Wylie', 'Freestyle', 100, 1912, 2), ('Jennie Fletcher', 'Freestyle', 100, 1912, 3), ('Kenneth Huszagh', 'Freestyle', 100, 1912, 3), ('George Ritchie Hodgson', 'Freestyle', 400, 1912, 1), ('John Gatenby Hatfield', 'Freestyle', 400, 1912, 2), ('Harold H. Hardwick', 'Freestyle', 400, 1912, 3), ('George Ritchie Hodgson', 'Freestyle', 1500, 1912, 1), ('John Gatenby Hatfield', 'Freestyle', 1500, 1912, 2), ('Harold H. Har

On va maintenant oublier que le rank a déjà été calculé et calculer cela nous-mêmes, on prendra quand même soin d'éliminer les tuples qui ont été disqualifiés pour prévenir tout résultat biaisé.

In [15]:
from sqlalchemy import text

session = engine.connect()

query = text("""
WITH QualifiedSoloAthletes AS (
    SELECT
        a.athlete_id, e.stroke, e.distance, e.year, a.name, r.results
    FROM athletes a
    INNER JOIN teams t ON a.athlete_id = t.athlete_id
    INNER JOIN events e ON t.event_id = e.event_id
    INNER JOIN results r ON t.team_id = r.team_id
    WHERE NOT e.is_relay
    AND r.rank > 0
)
SELECT 
    name, stroke, distance, year,
    RANK() OVER (PARTITION BY year, stroke, distance ORDER BY results ASC) AS rank
FROM QualifiedSoloAthletes
ORDER BY year, stroke, distance, rank
""")

result = session.execute(query)
print(result.fetchall())

session.close()

[('Harry J. Hebner', 'Backstroke', 100, 1912, 1), ('Otto Fahr', 'Backstroke', 100, 1912, 2), ('Paul Kellner', 'Backstroke', 100, 1912, 3), ('András Baronyi', 'Backstroke', 100, 1912, 4), ('Otto Gross', 'Backstroke', 100, 1912, 5), ('Walter Bathe', 'Breaststroke', 200, 1912, 1), ('Willy Lützow', 'Breaststroke', 200, 1912, 2), ('Kurt Paul Malisch', 'Breaststroke', 200, 1912, 3), ('Percy Courtman', 'Breaststroke', 200, 1912, 4), ('Walter Bathe', 'Breaststroke', 400, 1912, 1), ('Tor Henning', 'Breaststroke', 400, 1912, 2), ('Percy Courtman', 'Breaststroke', 400, 1912, 3), ('Kurt Paul Malisch', 'Breaststroke', 400, 1912, 4), ('Duke Paoa Kahanamoku', 'Freestyle', 100, 1912, 1), ('Cecil Healy', 'Freestyle', 100, 1912, 2), ('Kenneth Huszagh', 'Freestyle', 100, 1912, 3), ('Kurt Bretting', 'Freestyle', 100, 1912, 4), ('Walther Ramme', 'Freestyle', 100, 1912, 5), ('Fanny Durack', 'Freestyle', 100, 1912, 6), ('Mina Wylie', 'Freestyle', 100, 1912, 7), ('Jennie Fletcher', 'Freestyle', 100, 1912, 8),